In [3]:
import matplotlib
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import torch

#  SVM module
#from PredictSVM import PredictSVMChoice
# 
data_dir = '/media/cat/4TBSSD/yuki/'
#data_dir = '/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/'

# 
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [5]:
# ######################################################
# ##### PREDICT SVM DECISION CHOICE SINGLE SESSION #####
# ######################################################

# # select animal names
# names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2'] # "AR4" and other datasets could work
# names = ['AQ2']

# # 
# svm = PredictSVMChoice()
# svm.root_dir = data_dir
# svm.random_flag = False  # shuffle data to show baseline

# # window parameters
# svm.window = 30              # prediction window, backwards in time in seconds
# svm.lockout_window = 10      # time for locking out other pulls 
# svm.sliding_window = 30      # number of frames in sliding window

# # pca params
# svm.pca_flag = True
# svm.pca_var = 0.95  # this is default for now for nComp = 20
            
# # svm parameters
# svm.xvalidation = 10  # KFold xvalidation step
# svm.data_split = 0.8  # split 80/20 data for training
# svm.method = 'sigmoid'  # method used for computing SVM

# # run-time parameters
# svm.parallel = True
# svm.n_cores = svm.xvalidation
# svm.min_trials = 10
# svm.overwrite = True
# svm.code = 'code_04'

# # session info
# #session_ids = ['Mar1_', 'Mar2_', 'Mar3_', 'Feb29', 'Mar7_']
# svm.session_id = 'all'

# # 
# lockouts = [False, True]

In [103]:

class Net(nn.Module):
    
    def __init__(self, 
                 n_inputs,
                 n_hidden_nodes):    
        
        super(Net,self).__init__()
        
        #n_hidden_nodes = 50
        #Our network consists of 3 layers. 1 input, 1 hidden and 1 output layer
        
        #This applies Linear transformation to input data. 
        self.fc1 = nn.Linear(n_inputs,n_hidden_nodes)
        
        #This applies linear transformation to produce output data
        self.fc2 = nn.Linear(n_hidden_nodes, n_inputs)
        
    #This must be implemented
    def forward(self,x):
        
        #Output of the first layer
        x = self.fc1(x)
        
        #Activation function is Relu. Feel free to experiment with this
        x = F.tanh(x)
        
        #This produces output
        x = self.fc2(x)
        
        return x
        
    #This function takes an input and predicts the class, (0 or 1)        
    def predict(self,x):
        
        #Apply softmax to output
        pred = F.softmax(self.forward(x))
        ans = []
        for t in pred:
            if t[0]>t[1]:
                ans.append(0)
            else:
                ans.append(1)
        return torch.tensor(ans)
        
#
def predict(x):
    x = torch.from_numpy(x).type(torch.FloatTensor)
    ans = model.predict(x)
    
    return ans.numpy()

# Helper function to plot a decision boundary.
# If you don't fully understand this function don't worry, it just generates the contour plot below.
def plot_decision_boundary(pred_func,X,y):
    
    # Set min and max values and give it some padding
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = 0.01
    
    # Generate a grid of points with distance h between them
    xx,yy=np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    # Predict the function value for the whole gid
    Z = pred_func(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.binary)
    

def format_data(data, ratio, t):

    d1 = data[:int(data.shape[0]*ratio),:,t[0]:t[1]]
    train = d1.reshape(d1.shape[0],-1)

    d2 = data[int(data.shape[0]*ratio):,:,t[0]:t[1]]
    test = d2.reshape(d2.shape[0],-1)

    return train, test

def format_data_Kfold(data, t):

    d1 = data[:,:,t[0]:t[1]]
    d1 = d1.reshape(d1.shape[0],-1)

    return d1


def generate_training_data(r1, 
                           t1, 
                           t, 
                           ratio,
                          i):
    
    xvalidation = 10
    # shuffle
    idx = np.random.choice(np.arange(r1.shape[0]),
                           r1.shape[0],
                           replace=False)


    r1 = r1[ids]
    t1 = t1[idx]
    
    # get data
    r1, r2 = format_data(r1, ratio, t)
    t1, t2 = format_data(t1, ratio, t)
    X_train = np.vstack((r1,t1))
    X_test =  np.vstack((r2,t2))
    
    # get labels
    y_train = np.concatenate(
            (np.zeros(r1.shape[0]),
            np.ones(t1.shape[0])))
    y_test =  np.concatenate(
            (np.zeros(r2.shape[0]),
            np.ones(t2.shape[0])))
    
    return X_train, y_train, X_test, y_test


def generate_training_data_Kfold(random, 
                                 trials, 
                                 t, 
                                 ratio,
                                 i):
    
    xvalidation = 10
    # shuffle
    splits = np.array_split(np.arange(random.shape[0]), 
                            xvalidation)

    idx_test = splits[i]
    idx = np.delete(np.arange(random.shape[0]), idx_test)
        
    # 
    r1 = random[idx]
    r1_test = random[idx_test]
    t1 = trials[idx]
    t1_test = trials[idx_test]
    
    # get data
    r1 = format_data_Kfold(r1, t)
    t1 = format_data_Kfold(t1, t)
    
    # 
    r2 = format_data_Kfold(r1_test, t)
    t2 = format_data_Kfold(t1_test, t)
        
    # 
    X_train = np.vstack((r1,t1))
    X_test =  np.vstack((r2,t2))
    
    # get labels
    y_train = np.concatenate(
            (np.zeros(r1.shape[0]),
            np.ones(t1.shape[0])))
    y_test =  np.concatenate(
            (np.zeros(r2.shape[0]),
            np.ones(t2.shape[0])))
    
    return X_train, y_train, X_test, y_test


def train_model(X_train, 
                y_train,
                epochs,
                n_hidden_nodes):
    
    # 
    X = X_train
    y = y_train
    X = torch.from_numpy(X).type(torch.FloatTensor).cuda()
    y = torch.from_numpy(y).type(torch.LongTensor).cuda()


    #Initialize the model        
    n_inputs = X.shape[1]

    model = Net(n_inputs, n_hidden_nodes).cuda()

    #Define loss criterion
    criterion = nn.CrossEntropyLoss()

    #Define the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    #Number of epochs
#     epochs = 500

    #List to store losses
    losses = []
    for i in range(epochs):
        #Precit the output for Given input
        y_pred = model.forward(X)

        #Compute Cross entropy loss
        loss = criterion(y_pred,y)

        #Add loss to the list
        losses.append(loss.item())

        #Clear the previous gradients
        optimizer.zero_grad()

        #Compute gradients
        loss.backward()

        #Adjust weights
        optimizer.step()

    return model

In [104]:
import sklearn.datasets
import torch
import numpy as np
from tqdm import tqdm, trange

#
r1 = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar2_30Hz/IJ2pm_Mar2_30Hz_code_04_random_ROItimeCourses_30sec_pca_0.95.npy')
t1 = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar2_30Hz/IJ2pm_Mar2_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')
print (r1.shape, t1.shape)



(106, 6, 1801) (106, 6, 1801)


In [107]:
#
ratio = 0.8
epochs = 500
n_hidden_units = 200

# 
acc= np.zeros((900,10),'float32')

#
sliding_window = 10
window = 30
for k in trange(450,900,sliding_window):
    for i in range(10):

        # 
        t = [k,k+window]

        X_train, y_train, X_test, y_test = generate_training_data_Kfold(r1,
                                                                      t1,
                                                                      t,
                                                                      ratio,
                                                                      i)


        # 
        model = train_model(X_train, 
                            y_train,
                            epochs,
                            n_hidden_units)

        # predict on hold out data
        X1 = torch.from_numpy(X_test).type(torch.FloatTensor)
        y1 = torch.from_numpy(y_test).type(torch.FloatTensor)
        res = model.predict(X1.cuda())
        
        # 
        score = accuracy_score(res,y1)
        
        acc[k,i]= score
        



  0%|          | 0/45 [00:00<?, ?it/s]/media/cat/4TBSSD/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
<ipython-input-103-67ca0cabf815>:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = F.softmax(self.forward(x))
100%|██████████| 45/45 [11:04<00:00, 14.78s/it]


In [108]:
print (acc.shape)

(900, 10)


In [118]:
###############################################
############ COMPARE ALL 3 METHODS ############
###############################################

svm = np.load('/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2pm_Mar2_30Hzcode_04_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz')
acc1 = svm['accuracy']
print (acc1.shape)
svm_c = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar2_30Hz/IJ2pm_Mar2_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz')
acc2 = svm_c['accuracy']
print (acc2.shape)

acc1 = acc1[:900]
mean = acc1.mean(1)
std = np.std(acc1,1)
t=np.arange(acc1.shape[0])/30-30
plt.plot(t,mean, c='black')
plt.fill_between(t,mean+std, mean-std, color='black', alpha=.2,
                    label='single session')

acc2 = acc2[:900]
mean = acc2.mean(1)
std = np.std(acc2,1)
t=np.arange(acc2.shape[0])/30-29
plt.plot(t, mean, c='blue')
plt.fill_between(t,mean+std, mean-std, color='blue', alpha=.2,
                    label='concatenated')

# 
acc3 = acc
idx = np.where(acc3==0)[0]
acc3[idx]=np.nan
acc3=acc3[::10]
print (acc3.shape)
mean = acc3.mean(1)
std = np.std(acc3,1)
t=np.arange(acc3.shape[0])/3-30
plt.plot(t, mean, c='green')
plt.fill_between(t,mean+std, mean-std, color='green', alpha=.2,
                    label='single layer NN')

plt.plot([-30,0],[0.5,0.5],'r--',linewidth=3)
plt.legend()
plt.xlim(-15,0)
plt.show()


(1771, 10)
(870, 10)
(90, 10)


In [4]:
#####################################
##### PREDICT SVM DECISION TIME #####
#####################################
svm = PredictSVMTime()
svm.main_dir = '/media/cat/4TBSSD/yuki/'

# default params
svm.lockout = False
svm.lockout_window = 10
svm.pca_flag = True
svm.pca_var = 0.95
svm.window = 10
svm.use_saved_model = False
svm.fname_saved_model = '/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Feb11_30Hz/analysis/00599_svm.pkl'

# 
names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2'] # "AR4" and other datasets could work

# 
lockouts = [False, True]
for lockout in lockouts:
    svm.lockout=lockout
    for id_ in names:
        svm.animal_id = id_
        svm.session_id = 'all'

        # 
        svm.predict()

# svm.process_data()

  0%|          | 0/71 [00:00<?, ?it/s]

 too few trials... skipping


  4%|▍         | 3/71 [00:01<00:27,  2.46it/s]

no such filename, skipping
 too few trials... skipping


 14%|█▍        | 10/71 [00:02<00:13,  4.64it/s]

...

 30%|███       | 33/110 [00:05<00:19,  3.93it/s]

no such filename, skipping
no such filename, skipping
no such filename, skipping
no such filename, skipping
 too few trials... skipping
no such filename, skipping
no such filename, skipping
 too few trials... skipping
 too few trials... skipping
no such filename, skipping
no such filename, skipping
no such filename, skipping
no such filename, skipping
 too few trials... skipping


 62%|██████▏   | 68/110 [00:14<00:21,  1.97it/s]

no such filename, skipping


100%|██████████| 110/110 [00:35<00:00,  3.07it/s]


In [18]:
################################################
################################################
################################################
fig = plt.figure()
svm.animal_id = 'AQ2'
svm.session_id = 'all'
svm.get_sessions()
print (svm.sessions)

#
ctr=1
for session_id in svm.sessions:
    ax=plt.subplot(8,10,ctr)
    svm.session_id = session_id
    try:
        svm.plot_decision_time(ax)
        plt.title(session_id+", #: "+str(svm.n_trials), fontsize=6)
        if ctr!=1:
            plt.yticks([])
        if ctr!=31:
            plt.xticks([])

        ctr+=1

    except:
        pass
plt.suptitle(svm.animal_id,fontsize=20)

['AQ2am_Dec9_30Hz' 'AQ2am_Dec10_30Hz' 'AQ2pm_Dec10_30Hz'
 'AQ2am_Dec11_30Hz' 'AQ2pm_Dec14_30Hz' 'AQ2am_Dec14_30Hz'
 'AQ2pm_Dec16_30Hz' 'AQ2am_Dec17_30Hz' 'AQ2pm_Dec17_30Hz'
 'AQ2am_Dec18_30Hz' 'AQ2pm_Dec18_30Hz' 'AQ2am_Dec21_30Hz'
 'AQ2am_Dec22_30Hz' 'AQ2am_Dec23_30Hz' 'AQ2am_Dec28_30Hz'
 'AQ2am_Dec29_30Hz' 'AQ2am_Dec30_30Hz' 'AQ2am_Dec31_30Hz'
 'AQ2am_Jan4_30Hz' 'AQ2am_Jan5_30Hz' 'AQ2am_Jan6_30Hz' 'AQ2am_Jan7_30Hz'
 'AQ2am_Jan8_30Hz' 'AQ2pm_Jan11_30Hz' 'AQ2am_Jan11_30Hz'
 'AQ2pm_Jan12_30Hz' 'AQ2am_Jan12_30Hz' 'AQ2pm_Jan13_30Hz'
 'AQ2am_Jan13_30Hz' 'AQ2pm_Jan14_30Hz' 'AQ2am_Jan14_30Hz'
 'AQ2am_Jan15_30Hz' 'AQ2pm_Jan15_30Hz' 'AQ2pm_Jan18_30Hz'
 'AQ2am_Jan18_30Hz' 'AQ2am_Jan19_30Hz' 'AQ2pm_Jan19_30Hz'
 'AQ2pm_Jan20_30Hz' 'AQ2am_Jan20_30Hz' 'AQ2pm_Jan21_30Hz'
 'AQ2am_Jan21_30Hz' 'AQ2pm_Jan22_30Hz' 'AQ2am_Jan22_30Hz'
 'AQ2am_Jan25_30Hz' 'AQ2pm_Jan25_30Hz' 'AQ2pm_Jan26_30Hz'
 'AQ2am_Jan26_30Hz' 'AQ2am_Jan27_30Hz' 'AQ2pm_Jan27_30Hz'
 'AQ2am_Jan28_30Hz' 'AQ2pm_Jan28_30Hz' 'AQ2am_Jan29_30Hz'
 

Text(0.5, 0.98, 'AQ2')

In [ ]:
names = ['IA1','IA2','IA3','IJ1','AQ2'] # "AR4" and other datasets could work

for name in names:
    rnn.animal_id = name
    rnn.session_id = 'all'

    rnn.run_rnn()

In [12]:
#
data1 = np.load('/media/cat/4TBSSD/yuki/IJ2/RNN_scores/Mar3_200_0.0001.npy.npz')
data1 = np.load('/media/cat/4TBSSD/yuki/IJ2/RNN_scores/IJ2pm_Mar2_30Hz_1000_0.0001.npz')
#data1 = np.load('/media/cat/4TBSSD/yuki/IJ2/RNN_scores/IJ2pm_Feb29_30Hz_1000_0.0001.npz')
b = data1['b_rnn']
c = data1['c_s']

#
print (b.shape)
mean = b.mean(1)
t=np.linspace(-9.5, 0, mean.shape[0])

ax=plt.subplot(111)

plt.plot(t, mean, c='black')
plt.fill_between(t, mean+c, mean-c, color='black', alpha=.2,
                label='RNN - 1000epochs')

#
data1 = np.load('/media/cat/4TBSSD/yuki/IJ2/RNN_scores/IJ2pm_Mar2_30Hz_200_0.0001.npz')
#data1 = np.load('/media/cat/4TBSSD/yuki/IJ2/RNN_scores/IJ2pm_Feb29_30Hz_200_0.0001.npz')
b = data1['b_rnn']
c = data1['c_s']

#
print (b.shape)
mean = b.mean(1)
t=np.linspace(-9.5, 0, mean.shape[0])
plt.plot(t, mean, c='red')
plt.fill_between(t, mean+c, mean-c, color='red', alpha=.2,
                label='RNN - 200epochs')
#
if False:
    fname_svm = '/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2pm_Mar2_30Hz_code_04_trial_ROItimeCourses_10sec_pca_0.95.npy'
    data2 = np.load(fname_svm)
    print (data2.shape)
    data2 = data2[:300]
    mean = data2.mean(1)
    std = data2.std(1)
    t= np.linspace(t[0],t[-1],mean.shape[0])
    plt.plot(t, mean, 'blue')
    plt.fill_between(t, mean+std, mean-std, color='blue', alpha=.2,
                    label='SVM - 2 frames sliding window')

plt.legend(fontsize=20)
plt.xlim(-10,0)
plt.ylim(0.4, 1.0)
plt.plot([-10,0],[0.5,0.5],'r--',c='black')
ax.tick_params(axis='both', which='major', labelsize=20)
#plt.suptitle(os.path.split(fname_svm)[1])
plt.xlabel("Time (sec)",fontsize=20)
plt.ylabel("Decoding accuracy", fontsize=20)

plt.show()

(10, 10)
(10, 10)


<ipython-input-12-e7b69b6f7608>:48: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "r--" (-> color='r'). The keyword argument will take precedence.
  plt.plot([-10,0],[0.5,0.5],'r--',c='black')


In [ ]:
OLDER CODE BELOW

In [11]:
#############################################
############## VISUALIZE RESULTS ############
#############################################
    
# LEVER PULL
vis = Visualize()

# lever-related data
vis.main_dir = data_dir
vis.random_flag = False  # shuffle data to show baseline

# 
vis.window = 10
vis.lockout_window = 10
vis.code = 'code_04'
vis.lockout = False

vis.animal_id = "IA1"
vis.session_id = "Feb23_"
title = vis.animal_id + "  "+vis.session_id

#############################################
############## DECISION TYPE ################
#############################################
# select animal and session

# load pca 
fig =plt.figure()

# ax=plt.subplot(2,3,1)
# vis.pca_flag = False
# vis.lockout = True
# vis.pca_var = 0.99
# vis.plot_decision_choice('black',str(vis.pca_var),title, ax)

# vis.lockout = True
# vis.plot_decision_choice('blue',str(vis.pca_var)+" lockout",title, ax)

# 
# vis.lockout = True
# vis.plot_decision_choice('blue',str(vis.pca_var) + " lockout",title, ax)


# 
vars_ = [0.95]
for k in range(len(vars_)):
    #ax=plt.subplot(2,3,k+2)
    ax=plt.subplot(1,1,k+1)
    vis.pca_flag = True
    vis.lockout = False
    vis.pca_var = vars_[k]
    vis.plot_decision_choice('black',str(vis.pca_var),title, ax)

    # 
    vis.pca_flag = True
    vis.lockout = True
    vis.plot_decision_choice('blue',str(vis.pca_var)+" lockout",title, ax)

# vis.pca_flag = True
# vis.pca_var = 0.99
# ax=plt.subplot(2,3,2)
# vis.plot_decision_choice('blue','xxpcs_0.99Variance',title, ax)


# vis.pca_flag = True
# vis.pca_var = 0.95
# ax=plt.subplot(2,3,3)
# vis.plot_decision_choice('red','8pcs_0.95Variance',title, ax)

# vis.pca_flag = True
# vis.pca_var = 0.90
# ax=plt.subplot(2,3,4)
# vis.plot_decision_choice('green','4pcs_0.90Variance',title, ax)


# vis.pca_var = 0.50
# ax=plt.subplot(2,3,5)
# vis.plot_decision_choice('magenta','2pcs_0.50Variance',title, ax)


# fname = vis.main_dir+'/'+animal_id+'/SVM_scores_'+animal_id+'_lockout_'+str(session)+'.npy'
# vis.load_data(fname)
# vis.plot_decision_choice('blue','lockout-denoised')


Self session: IA1pm_Feb23_30Hz
Self session: IA1pm_Feb23_30Hz


In [7]:
import pickle as pk

file = open('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_10sec_pca.pkl', 'rb')

# dump information to that file
pca = pk.load(file)

print (pca.components_.shape)


(1500, 16384)


In [9]:
data_stm = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_10.0sec_butterworth_globalAverage_0.1hz_6.0hz_04code_stm.npy')
print (data_stm.shape)

(62, 601, 128, 128)


In [10]:
# 
X = data_stm.reshape(data_stm.shape[0]*data_stm.shape[1],
                     data_stm.shape[2]*data_stm.shape[3])

mu= np.mean(X, axis=0)


NameError: name 'nComp' is not defined

In [11]:
nComp = 9
Xnew = np.dot(pca.transform(X)[:,:nComp],
             pca.components_[:nComp,:])

Xnew+=mu

data_stm_denoised_reshaped = Xnew.reshape(data_stm.shape[0], data_stm.shape[1],
                                          data_stm.shape[2], data_stm.shape[3])


In [17]:
time_filters = pca.transform(X)[:,:nComp]
pca_time_filters_only = time_filters.reshape(data_stm.shape[0], data_stm.shape[1],-1).transpose(0,2,1)
print (pca_time_filters_only.shape)


(62, 9, 601)


(62, 601, 9)


In [21]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb23_30Hz/IA1pm_Feb23_30Hz_code_04_trial_ROItimeCourses_10sec_pca_0.5.npy')
print (data.shape)

(82, 2, 601)


In [76]:
# -*- coding: utf-8 -*-
"""Cat Data.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1r-NABuBJOITVlW03VvhsQivPNzmNp4yp
"""

import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold 
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn import svm
import pandas as pd
from sklearn.metrics import confusion_matrix  
import random
from scipy import stats
from matplotlib.pyplot import MultipleLocator
import os


jobid = os.getenv('SLURM_ARRAY_TASK_ID')
name='IA1_lockout'

data_leverpress = np.load('/media/cat/4TBSSD/yuki/temp/IA1/data_04.npy',allow_pickle=True)[0]
data_random = np.load('/media/cat/4TBSSD/yuki/temp/IA1/data_04_random.npy',allow_pickle=True)[0]

print (data_leverpress.shape)


(62, 35, 601)


In [77]:


#classification of time (10-class)
X_l=data_leverpress #  [n_trials, n_areas, n_time_steps]
X_l=X_l.transpose((0,2,1))


X_R_l= X_l.reshape(-1,X_l.shape[1]*X_l.shape[2])
normal_X_l = preprocessing.normalize(X_R_l)
n_X_l=normal_X_l.reshape(X_l.shape[0],X_l.shape[1],X_l.shape[2])
X_l=n_X_l

X_l=X_l[:,:300,:]
X_k=X_l[:,0:30,:]
X_k=X_k.reshape(X_k.shape[0],X_k.shape[1]*X_k.shape[2])
for i in range(30,271,30):
    X_t = X_l[:,i:i+30,:]
    X_t=X_t.reshape(X_t.shape[0],X_t.shape[1]*X_t.shape[2])
    X_k=np.concatenate((X_k,X_t),axis=0)

y_ct=np.zeros(data_leverpress.shape[0])
for i in range(1,10):
    Z_ct=i*np.ones(data_leverpress.shape[0])
    y_ct=np.concatenate((y_ct,Z_ct))

# 
X_tSVM=X_k # 10s * 30frams
y_tSVM=y_ct # labels

#10-fold confusion matrix
clf = svm.SVC() # Non-linear classifier
ten_svm=[]
conf_matrix_ten=[]
kf = KFold(n_splits=10,random_state=None, shuffle=True) 
for train_index, test_index in kf.split(X_tSVM):
    X_train_k, X_test_k = X_tSVM[train_index], X_tSVM[test_index] 
    y_train_k, y_test_k = y_tSVM[train_index], y_tSVM[test_index]
    clf.fit(X_train_k, y_train_k)
    score=clf.score(X_test_k, y_test_k)
    y_predicted=clf.predict(X_test_k)
    cm=confusion_matrix(y_test_k,y_predicted)
    confusion_m=cm.T # make each row be the prediction
    conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
    conf_matrix_norm = np.nan_to_num(conf_matrix_norm)
    ten_svm.append(score)
    conf_matrix_ten.append(conf_matrix_norm)
np.save('conf_10_'+str(name)+'_'+str(jobid)+'.npy',conf_matrix_ten)
sc=np.mean(conf_matrix_ten,axis=0) # mean of confusion matrix
np.save('conf_avg_'+str(name)+'_'+str(jobid)+'.npy',sc)

<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np

In [78]:

#Next is the plotting.

confusion_10=sc.diagonal()
np.save('diagonal_'+str(name)+'_'+str(jobid)+'.npy',confusion_10)
confusion_s=conf_matrix_ten

confusion_d=[]
for i in range(0,10):
    confusion_d.append(confusion_s[i].diagonal())
x_std=np.std(confusion_d,axis=0)/(10**0.5)
x_s=x_std


fig = plt.figure(figsize=(21.5,20), dpi=64, facecolor='white')
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
axes = plt.subplot(111)
axes.tick_params(axis ='both', which ='both', length = 10,width=4,pad=20)

bwith = 3 
ax = plt.gca()
ax.spines['bottom'].set_linewidth(bwith)
ax.spines['left'].set_linewidth(bwith)
ax.spines['top'].set_linewidth(bwith)
ax.spines['right'].set_linewidth(bwith)
l=15

Y=np.linspace(0,1,12)
X=np.ones(Y.size)
x=np.arange(-9.5,0.5,1)
plt.text(0.1, 0.2, 'Lever Pull', fontdict={'size': 70}, rotation=90)
plt.text(-12.95, 1/10, 'Chance', fontdict={'size': 60}, rotation=0)
plt.axhline(y=1/10, color='black', linestyle='--',linewidth=6)

plt.errorbar(x,confusion_10,x_s,c='red',alpha=0.6,marker='s', mec='red', ms=3, mew=3,label='SVM',linewidth=l,elinewidth=12)

plt.plot( (0+0)*X, Y, color='black',ls='--',linewidth=6)

plt.title('Decoding decision time',fontdict={'size': 110},pad=60)
plt.xlabel('Time before Lever Pull (s)',fontdict={'size': 110},labelpad=60)
plt.ylabel('Decoding Accuracy',fontdict={'size': 110},labelpad=180)
plt.ylim(ymax = 1.09)
ax=plt.gca()
ax.xaxis.set_major_locator(MultipleLocator(2))
ax.xaxis.set_minor_locator(MultipleLocator(1))
ax.yaxis.set_major_locator(MultipleLocator(0.2))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))

plt.xlim(xmin = -10.5)
plt.xlim(xmax = 0.9)

for i in range(0,10):
    a=confusion_10[i]
    b=x_s[i]
    n=10
    s=b
    t=(a-(1/10))/s
    df=n-1
    p = (1 - stats.t.cdf(t,df=df))
    
    if (p > 0.01 and p <= 0.05):
        plt.plot(-9.5+i, confusion_10[i]+x_std[i]+0.06, '*',c='black', ms=20)
        
    if (p > 0.00001 and p <= 0.01):
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.06,'*',c='black', ms=20)
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.08,'*',c='black', ms=20)
    if  p <= 0.00001:
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.06,'*',c='black', ms=20)
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.08,'*',c='black', ms=20)
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.10,'*',c='black', ms=20)

plt.show()
#plt.savefig('./time_svm_'+str(jobid)+'.jpg')
#plt.close()

<ipython-input-78-40a74f02ec5b>:57: RuntimeWarning: divide by zero encountered in double_scalars
  t=(a-(1/10))/s


In [10]:
data= np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar3_30Hz/IJ2pm_Mar3_30Hz_code_04_lockout_10sec_random_ROItimeCourses_10sec_pca_0.95_spatial.npy')
print (data.shape)

(7, 16384)
